In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import numpy as np

# 1. Dữ liệu huấn luyện nhỏ (chúng ta làm ví dụ đơn giản)
corpus = [
    "tôi yêu học máy",
    "tôi yêu học sâu",
    "tôi thích học python",
    "học máy rất thú vị",
    "học sâu rất mạnh mẽ",
    "học python rất dễ",
]

# 2. Tokenize văn bản
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1  # +1 cho padding

print("Từ vựng:", tokenizer.word_index)

# 3. Tạo dữ liệu huấn luyện dạng (input -> label)
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    # tạo các n-gram tăng dần
    for i in range(1, len(token_list)):
        n_gram = token_list[:i+1]
        input_sequences.append(n_gram)

# Padding cho cùng độ dài
max_seq_len = max(len(x) for x in input_sequences)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

# Tách dữ liệu đầu vào & nhãn (label = từ cuối cùng)
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# 4. Xây dựng mô hình RNN
model = Sequential([
    Embedding(total_words, 10, input_length=max_seq_len - 1),
    SimpleRNN(64),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# 5. Huấn luyện mô hình
model.fit(X, y, epochs=500, verbose=0)

# 6. Hàm dự đoán từ tiếp theo
def predict_next_word(seed_text, num_words=1):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break
    return seed_text

# 7. Thử nghiệm
print("\n➡️ Input: 'tôi yêu'")
print("➡️ Output:", predict_next_word("tôi yêu"))

print("\n➡️ Input: 'học'")
print("➡️ Output:", predict_next_word("học"))

print("\n➡️ Input: 'học máy rất'")
print("➡️ Output:", predict_next_word("học máy rất"))


Từ vựng: {'học': 1, 'tôi': 2, 'rất': 3, 'yêu': 4, 'máy': 5, 'sâu': 6, 'python': 7, 'thích': 8, 'thú': 9, 'vị': 10, 'mạnh': 11, 'mẽ': 12, 'dễ': 13}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


➡️ Input: 'tôi yêu'
➡️ Output: tôi yêu học

➡️ Input: 'học'
➡️ Output: học python

➡️ Input: 'học máy rất'
➡️ Output: học máy rất thú
